#深層距離学習を実装してみよう

##前回の復習
+ CIFAR-10（カラー）を題材に、オートエンコーダを使って異常検知
+ 正常：猫、異常：犬にして学習
+ オートエンコーダではCIFAR-10に太刀打ちできなかった（異常検知できなかった）

##今回の内容
+ 深層距離学習とは
+ 深層距離学習（L2-SoftmaxLoss）の論文説明
+ L2-SoftmaxLossをKerasで実装
+ Cifar-10を題材に「通常のCNN VS L2-SoftmaxLoss」の**分類精度**の比較


##前回で多かった質問
「オートエンコーダ」は、なぜカラー画像に対応できないのか？  
（回答）
+ そもそも、元画像と再構成画像を比べて異常検知するのは限界がある。
+ この手法は、細かい異常は捉えにくい。さらに背景に依存して異常度が変化する。
+ CIFAR-10の場合、犬と猫の違いは細かい所に集中している可能性がある（鼻や口など）。
+ さらに、多様な背景があるため、そこの部分に引っ張られ異常検知しにくい。
+ カラーの場合、3チャンネル分あるので、これらの要素がより大きく効いてくる。
+ モノクロ画像であっても、細かい異常や背景が多様だと対応できない。

（オートエンコーダの発展形）
+ 細かい異常を検知するために、小窓を用いる手法やSSIMという指標を用いる論文もある
+ https://qiita.com/shinmura0/items/811d01384e20bfd1e035
+ https://qiita.com/shinmura0/items/ee074172ec3c818b614e

#深層距離学習（Deep Metric Learning）とは
+ 元々は、分類問題で精度アップのために考えられた手法
+ 最近では、人物同定の際になくてはならない技術（それだけ人物同定の問題は難しい）
+ 似ているもの同士を近くに埋め込む手法
+ 上記の特性を生かし、異常画像は「似ていない」ため、外れた位置に埋め込まれ、異常検知可能となる

#L2-SoftmaxLossの論文
https://qiita.com/shinmura0/items/0d6685e378a57ca97a91

#L2-SoftmaxLossの実装
##data load

In [0]:
from keras.datasets import cifar10
import numpy as np
import matplotlib.pyplot as plt
from keras import backend as K
from keras.utils import to_categorical
from keras.applications import MobileNetV2
from keras.layers import Input, GlobalAveragePooling2D
from keras.layers import Dense, Activation
from keras.models import Model
import keras
from keras.optimizers import Adam

def get_cifar_data():
    # dataset
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()

    x_train = x_train.astype('float32') / 255
    x_test = x_test.astype('float32') / 255
        
    return x_train, x_test, y_train, y_test

Using TensorFlow backend.


In [0]:
x_train, x_test, y_train, y_test = get_cifar_data()

170500096/170498071 [==============================] - 13s 0us/step


##CNN

In [0]:
def get_L2(x, y, classes=10):
    print("L2-SoftmaxLoss")
    mobile = MobileNetV2(include_top=False, input_shape=x.shape[1:], alpha=0.5,
                         weights=None)
    
    # L2層と全結合層を付ける
    c = GlobalAveragePooling2D()(mobile.input)
    c = keras.layers.Lambda(lambda xx: 15*(xx)/K.sqrt(K.sum(xx**2)))(c) # L2-SoftmaxLoss
    c = Dense(classes, activation='softmax')(c)
    model = Model(inputs=mobile.input,outputs=c)

    #model.summary()

    model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(lr=0.0001, amsgrad=True),
                  metrics=['accuracy'])
    
    return model

def get_Normal(x, y, classes=10):
    print("Normal CNN")
    mobile = MobileNetV2(include_top=False, input_shape=x.shape[1:], alpha=0.5,
                         weights=None)
    
    # 全結合層を付ける
    c = GlobalAveragePooling2D()(mobile.output)
    c = Dense(classes, activation='softmax')(c)
    model = Model(inputs=mobile.output,outputs=c)

    #model.summary()

    model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(lr=0.0001, amsgrad=True),
                  metrics=['accuracy'])
    
    return model

def train(model, x, y, x_val, y_val):
    #学習
    Y = to_categorical(y)
    Y_val = to_categorical(y_val)
    hist = model.fit(x, Y,
                     validation_data=(x_val, Y_val),
                     batch_size=128, epochs=1000, verbose = False)

    plt.figure()               
    plt.plot(hist.history['val_acc'],label="val_acc")
    plt.legend(loc="lower right")
    plt.show()

    print("Validation Accuracy",np.max(hist.history['val_acc']))

    return model

##Normal CNN

In [0]:
model = get_Normal(x_train, y_train)
model = train(model, x_train, y_train, x_test, y_test)

##L2-SoftmaxLoss

In [0]:
model = get_L2(x_train, y_train)
model = train(model, x_train, y_train, x_test, y_test)